# Scoring Simulator

This notebook allows you to simulate scoring for a conversation by providing:
- `conversation_id`: The UUID of the conversation to score
- `user_id`: The UUID of the user (for verification)

It will compute and display:
1. **Conversation Scores**: Fillerwords, Clarity, Participation, Key Themes, Index of Questions, Rhythm, and Objective
2. **Profile Scores**: Prospection, Empathy, Technical Domain, Negotiation, and Resilience

In [1]:
# Import necessary libraries and services
import asyncio
import sys
import os
import time
import numpy as np
import pandas as pd
from pathlib import Path
from uuid import UUID
from dotenv import load_dotenv
from IPython.display import display, HTML
from collections import defaultdict

# Add parent directory to path to import app modules
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Load environment variables
load_dotenv()

# Import services
from app.services.messages_service import get_conversation_transcript
from app.services.conversations_service import get_conversation_details
from app.services.db import execute_query_one
from scoring_scripts.get_conver_scores import get_conver_scores
from scoring_scripts.get_conver_skills import get_conver_skills

print("✅ Imports successful!")

✅ Imports successful!


## Auxiliary functions

In [5]:
async def get_conversation_with_stage(conversation_id: UUID):
    """Get conversation details including course_id and stage_id with all feedback"""
    query = """
    SELECT 
        c.conversation_id, 
        c.user_id,
        c.course_id, 
        c.stage_id,
        c.start_timestamp, 
        c.end_timestamp, 
        c.status,
        sbc.general_score,
        sbc.fillerwords_scoring,
        sbc.clarity_scoring,
        sbc.participation_scoring,
        sbc.keythemes_scoring,
        sbc.indexofquestions_scoring,
        sbc.rhythm_scoring,
        sbc.is_accomplished,
        sbc.fillerwords_feedback,
        sbc.clarity_feedback,
        sbc.participation_feedback,
        sbc.keythemes_feedback,
        sbc.indexofquestions_feedback,
        sbc.rhythm_feedback, 
        cs.stage_objectives
    FROM conversaApp.conversations c
    LEFT JOIN conversaapp.scoring_by_conversation sbc ON c.conversation_id = sbc.conversation_id
    LEFT JOIN conversaconfig.course_stages cs ON c.course_id = cs.course_id AND c.stage_id = cs.stage_id
    WHERE c.conversation_id = $1
    """
    
    result = await execute_query_one(query, conversation_id)
    return dict(result) if result else None


In [6]:
async def fetch_conversation_details(conversation_id: UUID, user_id: str = None):
    """Fetch conversation details and return them"""
    conversation_details = await get_conversation_with_stage(conversation_id)

    if not conversation_details:
        print(f"❌ Conversation {conversation_id} not found!")
        raise ValueError("Conversation not found")

    # Extract required IDs
    course_id = conversation_details.get("course_id")
    stage_id = conversation_details.get("stage_id")
    conv_user_id = conversation_details.get("user_id")
    stage_objectives = conversation_details.get("stage_objectives")

    print(f"📋 Conversation Details:")
    print(f"   User ID: {conv_user_id}")
    print(f"   Course ID: {course_id}")
    print(f"   Stage ID: {stage_id}")
    print(f"   Status: {conversation_details.get('status')}")
    print(f"   Start: {conversation_details.get('start_timestamp')}")
    print(f"   End: {conversation_details.get('end_timestamp')}")
    print(f"   Stage Objectives: {stage_objectives}")

    # Verify user_id if provided
    if user_id and str(conv_user_id) != user_id:
        print(f"⚠️  Warning: Provided user_id ({user_id}) doesn't match conversation user_id ({conv_user_id})")

    if not course_id or not stage_id:
        print("❌ Missing course_id or stage_id in conversation!")
        raise ValueError("Conversation missing required course_id or stage_id")
    
    return {
        'conversation_details': conversation_details,
        'course_id': course_id,
        'stage_id': stage_id,
        'user_id': conv_user_id,
        'stage_objectives': stage_objectives
    }

In [7]:
def display_transcript(transcript):
    if not transcript:
        print("❌ No transcript found for this conversation!")
    else:
        # Prepare data for table
        transcript_data = []
        for idx, turn in enumerate(transcript, 1):
            speaker = turn.get('speaker', 'Unknown')
            text = turn.get('text', '')
            duration = turn.get('duracion', 'N/A')
            
            # Format speaker name
            speaker_display = "👤 Vendedor" if speaker == "vendedor" else "🤖 Cliente"
            
            # Count words
            word_count = len(text.split()) if text else 0
            
            transcript_data.append({
                '#': idx,
                'Speaker': speaker_display,
                'Text': text,
                'Words': word_count,
                'Duration (s)': duration if isinstance(duration, (int, float)) else 'N/A'
            })
        
        # Create DataFrame
        df_transcript = pd.DataFrame(transcript_data)
        
        # Calculate summary statistics
        total_turns = len(transcript_data)
        vendedor_turns = sum(1 for t in transcript_data if 'Vendedor' in t['Speaker'])
        cliente_turns = sum(1 for t in transcript_data if 'Cliente' in t['Speaker'])
        total_words = sum(t['Words'] for t in transcript_data)
        vendedor_words = sum(t['Words'] for t in transcript_data if 'Vendedor' in t['Speaker'])
        cliente_words = sum(t['Words'] for t in transcript_data if 'Cliente' in t['Speaker'])
        
        # Display summary
        print("=" * 100)
        print("📝 CONVERSATION TRANSCRIPT")
        print("=" * 100)
        print(f"\n📊 Summary:")
        print(f"   Total Turns: {total_turns}")
        print(f"   Vendedor Turns: {vendedor_turns} ({vendedor_turns/total_turns*100:.1f}%)")
        print(f"   Cliente Turns: {cliente_turns} ({cliente_turns/total_turns*100:.1f}%)")
        print(f"   Total Words: {total_words}")
        print(f"   Vendedor Words: {vendedor_words} ({vendedor_words/total_words*100:.1f}%)")
        print(f"   Cliente Words: {cliente_words} ({cliente_words/total_words*100:.1f}%)")
        print()
        
        # Display transcript table with HTML for better formatting
        display(HTML(df_transcript.to_html(
            index=False, 
            escape=False, 
            classes='table table-striped table-hover',
            table_id='transcript_table'
        )))

In [8]:
async def show_conver_scores(transcript, course_id, stage_id):
    """Compute and display conversation scores"""
    print("🔄 Computing conversation scores...")
    print("=" * 60)

    # Compute scores directly using the transcript passed as parameter
    scoring_results = await get_conver_scores(transcript, course_id, stage_id)

    # Extract scores and feedback from the results
    conversation_scores_direct = {
        'general_score': scoring_results.get('puntuacion_global', 'N/A'),
        'fillerwords_scoring': scoring_results.get('detalle', {}).get('muletillas_pausas', 'N/A'),
        'clarity_scoring': scoring_results.get('detalle', {}).get('claridad', 'N/A'),
        'participation_scoring': scoring_results.get('detalle', {}).get('participacion', 'N/A'),
        'keythemes_scoring': scoring_results.get('detalle', {}).get('cobertura', 'N/A'),
        'indexofquestions_scoring': scoring_results.get('detalle', {}).get('preguntas', 'N/A'),
        'rhythm_scoring': scoring_results.get('detalle', {}).get('ppm', 'N/A'),
        'is_accomplished': scoring_results.get('objetivo', {}),
        'fillerwords_feedback': scoring_results.get('feedback', {}).get('muletillas_pausas', 'N/A'),
        'clarity_feedback': scoring_results.get('feedback', {}).get('claridad', 'N/A'),
        'participation_feedback': scoring_results.get('feedback', {}).get('participacion', 'N/A'),
        'keythemes_feedback': scoring_results.get('feedback', {}).get('cobertura', 'N/A'),
        'indexofquestions_feedback': scoring_results.get('feedback', {}).get('preguntas', 'N/A'),
        'rhythm_feedback': scoring_results.get('feedback', {}).get('ppm', 'N/A'),
        'objetivo': scoring_results.get('objetivo', {}),
    }

    # Store in global variable for use in other cells
    global updated_scores
    updated_scores = conversation_scores_direct

    print("\n" + "=" * 60)
    print("📊 CONVERSATION SCORES SUMMARY (COMPUTED)")
    print("=" * 60)
    print(f"General Score: {updated_scores.get('general_score', 'N/A')}")
    print(f"Fillerwords: {updated_scores.get('fillerwords_scoring', 'N/A')}")
    print(f"Clarity: {updated_scores.get('clarity_scoring', 'N/A')}")
    print(f"Participation: {updated_scores.get('participation_scoring', 'N/A')}")
    print(f"Key Themes: {updated_scores.get('keythemes_scoring', 'N/A')}")
    print(f"Index of Questions: {updated_scores.get('indexofquestions_scoring', 'N/A')}")
    print(f"Rhythm: {updated_scores.get('rhythm_scoring', 'N/A')}")
    print(f"Objective Accomplished: {updated_scores.get('is_accomplished', 'N/A')}")

    print("\n📝 FEEDBACK:")
    print(f"Fillerwords: {updated_scores.get('fillerwords_feedback', 'N/A')[:150]}...")
    print(f"Clarity: {updated_scores.get('clarity_feedback', 'N/A')[:150]}...")
    print(f"Participation: {updated_scores.get('participation_feedback', 'N/A')[:150]}...")
    print(f"Key Themes: {updated_scores.get('keythemes_feedback', 'N/A')[:150]}...")
    print(f"Index of Questions: {updated_scores.get('indexofquestions_feedback', 'N/A')[:150]}...")
    print(f"Rhythm: {updated_scores.get('rhythm_feedback', 'N/A')[:150]}...")
    
    return updated_scores

async def show_conver_profiles(transcript):
    """Compute and display profile scores"""
    # Declare global variable first
    global profiling_results
    
    print("🔄 Computing profile scores...")
    print("=" * 60)

    # Compute profiles directly using the transcript passed as parameter
    profiling_results_computed = await get_conver_skills(transcript)

    # Extract scores and feedback from the computed results
    profiling_results = {
        'prospection_scoring': profiling_results_computed.get('prospection', {}).get('score', 'N/A'),
        'empathy_scoring': profiling_results_computed.get('empathy', {}).get('score', 'N/A'),
        'technical_domain_scoring': profiling_results_computed.get('technical_domain', {}).get('score', 'N/A'),
        'negotiation_scoring': profiling_results_computed.get('negociation', {}).get('score', 'N/A'),
        'resilience_scoring': profiling_results_computed.get('resilience', {}).get('score', 'N/A'),
        'prospection_feedback': profiling_results_computed.get('prospection', {}).get('justification', 'N/A'),
        'empathy_feedback': profiling_results_computed.get('empathy', {}).get('justification', 'N/A'),
        'technical_domain_feedback': profiling_results_computed.get('technical_domain', {}).get('justification', 'N/A'),
        'negotiation_feedback': profiling_results_computed.get('negociation', {}).get('justification', 'N/A'),
        'resilience_feedback': profiling_results_computed.get('resilience', {}).get('justification', 'N/A'),
    }

    print("\n" + "=" * 60)
    print("👥 PROFILE SCORES SUMMARY (COMPUTED)")
    print("=" * 60)
    print(f"Prospection: {profiling_results.get('prospection_scoring', 'N/A')}")
    print(f"Empathy: {profiling_results.get('empathy_scoring', 'N/A')}")
    print(f"Technical Domain: {profiling_results.get('technical_domain_scoring', 'N/A')}")
    print(f"Negotiation: {profiling_results.get('negotiation_scoring', 'N/A')}")
    print(f"Resilience: {profiling_results.get('resilience_scoring', 'N/A')}")

    print("\n📝 FEEDBACK:")
    print(f"Prospection: {profiling_results.get('prospection_feedback', 'N/A')[:150]}...")
    print(f"Empathy: {profiling_results.get('empathy_feedback', 'N/A')[:150]}...")
    print(f"Technical Domain: {profiling_results.get('technical_domain_feedback', 'N/A')[:150]}...")
    print(f"Negotiation: {profiling_results.get('negotiation_feedback', 'N/A')[:150]}...")
    print(f"Resilience: {profiling_results.get('resilience_feedback', 'N/A')[:150]}...")
    
    return profiling_results

In [9]:
def display_results(): 

    # Prepare comprehensive data with scores and feedback
    conversation_data = []
    conversation_metrics = [
        ("General Score", "general_score", None),
        ("Fillerwords", "fillerwords_scoring", "fillerwords_feedback"),
        ("Clarity", "clarity_scoring", "clarity_feedback"),
        ("Participation", "participation_scoring", "participation_feedback"),
        ("Key Themes", "keythemes_scoring", "keythemes_feedback"),
        ("Index of Questions", "indexofquestions_scoring", "indexofquestions_feedback"),
        ("Rhythm", "rhythm_scoring", "rhythm_feedback"),
        ("Objective Accomplished", "is_accomplished", None)
    ]

    for metric_name, score_key, feedback_key in conversation_metrics:
        score_value = updated_scores.get(score_key, 'N/A')
        if score_key == "is_accomplished":
            score_value = "Yes" if score_value else "No"
        feedback_value = updated_scores.get(feedback_key, 'N/A') if feedback_key else 'N/A'
        conversation_data.append({
            "Category": "Conversation",
            "Metric": metric_name,
            "Score": score_value,
            "Feedback": feedback_value if feedback_value != 'N/A' else '-'
        })

    # Prepare profile data with scores and feedback
    profile_data = []
    profile_skills = [
        ("Prospection", "prospection_scoring", "prospection_feedback"),
        ("Empathy", "empathy_scoring", "empathy_feedback"),
        ("Technical Domain", "technical_domain_scoring", "technical_domain_feedback"),
        ("Negotiation", "negotiation_scoring", "negotiation_feedback"),
        ("Resilience", "resilience_scoring", "resilience_feedback")
    ]

    if profiling_results:
        for skill_name, score_key, feedback_key in profile_skills:
            score_value = profiling_results.get(score_key, 'N/A')
            feedback_value = profiling_results.get(feedback_key, 'N/A')
            profile_data.append({
                "Category": "Profile",
                "Metric": skill_name,
                "Score": score_value,
                "Feedback": feedback_value if feedback_value != 'N/A' else '-'
            })

    # Combine all data
    all_data = conversation_data + profile_data

    # Create comprehensive DataFrame
    df_all = pd.DataFrame(all_data)

    # Configure pandas display options for better formatting
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_columns', None)

    # Display the comprehensive table
    print("=" * 100)
    print("📊 COMPLETE SCORING RESULTS WITH FEEDBACK")
    print("=" * 100)
    print()

    # Display with HTML for better formatting in Jupyter
    display(HTML(df_all.to_html(index=False, escape=False, classes='table table-striped table-hover')))


    # Create separate tables for better readability
    print("\n" + "=" * 100)
    print("🎯 CONVERSATION SCORES:")
    print("=" * 100)
    df_conv = pd.DataFrame(conversation_data)
    display(HTML(df_conv.to_html(index=False, escape=False)))

    print("\n" + "=" * 100)
    print("👥 PROFILE SCORES:")
    print("=" * 100)
    if profile_data:
        df_prof = pd.DataFrame(profile_data)
        display(HTML(df_prof.to_html(index=False, escape=False)))
    else:
        print("No profile scores available")

    print("\n" + "=" * 100)

In [18]:
async def stress_test(iterations=10, transcript=None):

    print("🧪 STRESS TEST: Computing scores 5 times...")
    print("=" * 100)
    print("This will help us understand the variance in LLM-based scoring.\n")

    # Store all runs
    all_conversation_runs = []
    all_profile_runs = []

    # Run 5 iterations
    num_runs = iterations
    for run_num in range(1, num_runs + 1):
        print(f"🔄 Run {run_num}/{num_runs}...")
        start_time = time.time()
        
        # Compute conversation scores
        scoring_results = await get_conver_scores(transcript, COURSE_ID, STAGE_ID)
        
        # Extract conversation scores
        conv_run = {
            'run': run_num,
            'general_score': scoring_results.get('puntuacion_global', 'N/A'),
            'fillerwords_scoring': scoring_results.get('detalle', {}).get('muletillas_pausas', 'N/A'),
            'clarity_scoring': scoring_results.get('detalle', {}).get('claridad', 'N/A'),
            'participation_scoring': scoring_results.get('detalle', {}).get('participacion', 'N/A'),
            'keythemes_scoring': scoring_results.get('detalle', {}).get('cobertura', 'N/A'),
            'indexofquestions_scoring': scoring_results.get('detalle', {}).get('preguntas', 'N/A'),
            'rhythm_scoring': scoring_results.get('detalle', {}).get('ppm', 'N/A'),
            'is_accomplished': scoring_results.get('objetivo', {}),
        }
        all_conversation_runs.append(conv_run)
        
        # Compute profile scores
        profiling_results_computed = await get_conver_skills(transcript)
        
        # Extract profile scores
        prof_run = {
            'run': run_num,
            'prospection_scoring': profiling_results_computed.get('prospection', {}).get('score', 'N/A'),
            'empathy_scoring': profiling_results_computed.get('empathy', {}).get('score', 'N/A'),
            'technical_domain_scoring': profiling_results_computed.get('technical_domain', {}).get('score', 'N/A'),
            'negotiation_scoring': profiling_results_computed.get('negociation', {}).get('score', 'N/A'),
            'resilience_scoring': profiling_results_computed.get('resilience', {}).get('score', 'N/A'),
        }
        all_profile_runs.append(prof_run)
        
        elapsed = time.time() - start_time
        print(f"   ✅ Completed in {elapsed:.2f}s\n")

    print("=" * 100)
    print("✅ All 5 runs completed!")
    print("=" * 100)

    return all_conversation_runs, all_profile_runs

In [11]:
def display_stress_test(all_conversation_runs, all_profile_runs):

    df_conv_runs = pd.DataFrame(all_conversation_runs)
    df_prof_runs = pd.DataFrame(all_profile_runs)

    # Calculate statistics for each metric
    def calculate_stats(values):
        """Calculate statistics for a list of numeric values"""
        numeric_values = [v for v in values if isinstance(v, (int, float)) and not isinstance(v, bool)]
        if not numeric_values:
            return {'mean': 'N/A', 'std': 'N/A', 'min': 'N/A', 'max': 'N/A', 'range': 'N/A'}
        
        mean_val = np.mean(numeric_values)
        std_val = np.std(numeric_values)
        min_val = np.min(numeric_values)
        max_val = np.max(numeric_values)
        range_val = max_val - min_val
        
        return {
            'mean': round(mean_val, 2),
            'std': round(std_val, 2),
            'min': min_val,
            'max': max_val,
            'range': round(range_val, 2)
        }

    # Conversation scores statistics
    conv_stats = {}
    conv_metrics = [
        col for col in df_conv_runs.columns 
        if any(key in col for key in [
            'general_score', 'fillerwords_scoring', 'clarity_scoring', 
            'participation_scoring', 'keythemes_scoring', 'indexofquestions_scoring', 
            'rhythm_scoring'
        ])
    ]
    for metric in conv_metrics:
        # skip if not present
        if metric not in df_conv_runs:
            continue
        values = df_conv_runs[metric].tolist()
        conv_stats[metric] = calculate_stats(values)

    # Profile scores statistics
    prof_stats = {}
    prof_metrics = [
        col for col in df_prof_runs.columns
        if any(key in col for key in [
            'prospection_scoring', 'empathy_scoring', 'technical_domain_scoring',
            'negotiation_scoring', 'resilience_scoring'
        ])
    ]
    for metric in prof_metrics:
        if metric not in df_prof_runs:
            continue
        values = df_prof_runs[metric].tolist()
        prof_stats[metric] = calculate_stats(values)

    # Display all 5 runs side by side
    print("\n" + "=" * 100)
    print("📋 ALL RUNS - CONVERSATION SCORES")
    print("=" * 100)
    display(HTML(df_conv_runs.to_html(index=False, escape=False, classes='table table-striped table-hover')))

    print("\n" + "=" * 100)
    print("📋 ALL RUNS - PROFILE SCORES")
    print("=" * 100)
    display(HTML(df_prof_runs.to_html(index=False, escape=False, classes='table table-striped table-hover')))

## Input Parameters

Enter the `conversation_id` and `user_id` below:

In [4]:
# Input parameters
CONVERSATION_ID = "5376d207-fcc4-4bd0-ae5b-b70fed2e9040"  # Replace with your conversation_id
USER_ID = "your-user-id-here"  # Replace with your user_id

# Validate UUIDs
try:
    conv_uuid = UUID(CONVERSATION_ID)
    user_uuid = UUID(USER_ID) if USER_ID != "your-user-id-here" else None
    print(f"✅ Conversation ID: {CONVERSATION_ID}")
    if user_uuid:
        print(f"✅ User ID: {USER_ID}")
    else:
        print("⚠️  User ID not provided (will be extracted from conversation)")
except ValueError as e:
    print(f"❌ Invalid UUID format: {e}")
    raise

✅ Conversation ID: 5376d207-fcc4-4bd0-ae5b-b70fed2e9040
⚠️  User ID not provided (will be extracted from conversation)


## Get Conversation Details

First, we'll fetch the conversation details to get `course_id` and `stage_id`:

In [12]:
# Fetch conversation details using await (Jupyter supports top-level await)
conv_info = await fetch_conversation_details(conv_uuid, USER_ID if USER_ID != "your-user-id-here" else None)

# Extract the IDs for use in subsequent cells
COURSE_ID = conv_info['course_id']
STAGE_ID = conv_info['stage_id']
CONV_USER_ID = conv_info['user_id']
conversation_details = conv_info['conversation_details']

📋 Conversation Details:
   User ID: ae1ed3b5-0ea0-4ad5-9565-fec1e4ef1ee7
   Course ID: 3eeeda53-7dff-40bc-b036-b608acb89e6f
   Stage ID: afa3a709-7c7b-4a64-9642-3d6eec19cc53
   Status: FINISHED
   Start: 2026-02-09 17:24:46.480844
   End: 2026-02-09 17:29:02.284048
   Stage Objectives: Cerrar un presupuesto argumentando sobre Estabilidad, Metalurgia y Normativa:

- Argumento de Estabilidad (Pulsado/Doble Pulsado): Explicar cómo el modo pulsado y las curvas sinérgicas de la máquina reducen las proyecciones y el retrabajo, garantizando uniformidad, vital para sus inspecciones de ultrasonidos

- Argumento de Alimentación (4 Ruletas): Detallar técnicamente el sistema de arrastre de 4 ruletas para garantizar una velocidad de hilo constante (24 m/min) , eliminando micro-cortes en el arco, especialmente en Aluminio.

- Argumento de Calibración y Servicio: Vender no solo la máquina, sino el servicio de "Calibración o verificación" que ofrece Gala Gar para el cumplimiento de normativa vigente, 

## Conversation Transcript

Displaying the full conversation transcript:

In [13]:
# Get the conversation transcript
transcript = await get_conversation_transcript(conv_uuid)
display_transcript(transcript)

📝 CONVERSATION TRANSCRIPT

📊 Summary:
   Total Turns: 15
   Vendedor Turns: 7 (46.7%)
   Cliente Turns: 8 (53.3%)
   Total Words: 567
   Vendedor Words: 194 (34.2%)
   Cliente Words: 373 (65.8%)



#,Speaker,Text,Words,Duration (s)
1,🤖 Cliente,"¡Hola!, ¿qué tal?",3,N/A
2,👤 Vendedor,"Buenos días, Roberto.",3,8.9
3,🤖 Cliente,Buenos días. ¿En qué puedo ayudarte hoy?,7,N/A
4,👤 Vendedor,"Mire, soy Daniel de Galagar. Está en una videoconferencia técnica con el responsable de calidad de una empresa que fabrica estructuras críticas. Nuestro objetivo va a ser cerrar un presupuesto argumentando los puntos fuertes de nuestras soldaduras.",37,17.417
5,🤖 Cliente,"Entendido, Daniel. Como ingeniero de SoldaQueen, mi principal preocupación es la estabilidad del proceso y la certificación ISO. Necesito asegurarme de que cualquier equipo que consideremos cumpla con los estándares más exigentes. Estoy listo para revisar los detalles técnicos y ver cómo vuestras soldaduras pueden integrarse en nuestros procesos.",49,N/A
6,👤 Vendedor,"Perfecto, te cuento. El modo pulsado de Galagar y las curvas sinérgicas de la máquina reducen las proyecciones y el retrabajo, garantizando uniformidad, vital para las extensiones de ultrasonidos.",29,13.754
7,🤖 Cliente,"Eso suena interesante, Daniel. La reducción de proyecciones y el retrabajo son cruciales para la eficiencia y la calidad de nuestras estructuras. Sin embargo, tengo una pregunta importante. Necesito trabajar en turnos intensivos. ¿Realmente la máquina de 500A aguanta o se calienta? Me preocupa la estabilidad del proceso en condiciones de uso continuo.",53,N/A
8,👤 Vendedor,"La estabilidad es cojonuda, Roberto, te cuento. El sistema de pulsado tiene un factor de marcha al noventa por ciento a quinientos amperios.",23,45.512
9,🤖 Cliente,"Eso es excelente, Daniel. Un factor de marcha al noventa por ciento a quinientos amperios es muy adecuado para turnos intensivos. Además, me gustaría saber cómo manejan el tema del aluminio, ya que hemos tenido problemas con la alimentación del hilo en nuestras máquinas actuales.",45,N/A
10,👤 Vendedor,"En cuanto a la alimentación, Roberto, tenemos lo mejor de la competencia. El detalle técnico se basa en el sistema del arrastre de cuatro ruletas que garantiza una velocidad de hidro constante de veinticuatro metros al minuto, eliminando microcortes en el arco y especialmente en aluminio, que comentas.",48,18.669


## Compute Conversation Scores

Computing all conversation scores using `scoring_service`:

In [14]:
# Compute and display conversation scores
updated_scores = await show_conver_scores(transcript, COURSE_ID, STAGE_ID)

🔄 Computing conversation scores...

📊 CONVERSATION SCORES SUMMARY (COMPUTED)
General Score: 88.5
Fillerwords: 85
Clarity: 60
Participation: 90
Key Themes: 100
Index of Questions: 0
Rhythm: 20
Objective Accomplished: True

📝 FEEDBACK:
Fillerwords: El porcentaje de muletillas empleadas es 1.55%, siendo las muletillas mas repetidas: mire, va ...
Clarity: Deberías enfocarte en mantener ideas estructuradas y claras en tus explicaciones. Usa términos específicos y evita frases demasiado largas que puedan ...
Participation: Tu porcentaje de participación ha sido del 34.22%. Has mostrado escucha activa en 4 ocasiones....
Key Themes: Has cubierto todos los temas clave esenciales de calidad de soldadura, precisión técnica y cumplimiento normativo. Excelente trabajo en desarrollar ca...
Index of Questions: Métrica desactivada temporalmente...
Rhythm: Velocidad de habla demasiado lenta (81.4 PPM). Intenta aumentar el ritmo para mantener la atención del cliente....


## Compute Profile Scores

Computing all profile scores using `profiling_service`:

In [15]:
profiling_results = await show_conver_profiles(transcript)

🔄 Computing profile scores...

👥 PROFILE SCORES SUMMARY (COMPUTED)
Prospection: 5
Empathy: 5
Technical Domain: 4
Negotiation: 5
Resilience: 5

📝 FEEDBACK:
Prospection: Demuestras investigación especifica al conocer detalles técnicos y necesidades del cliente, como 'certificación ISO' y 'problemas con aluminio', ademá...
Empathy: No interrumpes y validas las preocupaciones del cliente con frases como 'Eso suena interesante' y 'Eso es excelente', mostrando empatía y comprensión ...
Technical Domain: El vendedor menciona 'el sistema del arrastre de cuatro ruletas que garantiza una velocidad de hidro constante de veinticuatro metros al minuto', demo...
Negotiation: El vendedor justifica el valor destacando beneficios técnicos, como 'reduce proyecciones y retrabajo' y 'garantiza cumplimiento de normativa', y propo...
Resilience: El vendedor mantiene un tono positivo y enérgico, responde con seguridad a las objeciones del cliente, recupera la energía tras cada respuesta (como e...


## Complete Results Summary

Displaying all scores in a formatted table:

In [16]:
display_results()

📊 COMPLETE SCORING RESULTS WITH FEEDBACK



Category,Metric,Score,Feedback
Conversation,General Score,88.5,-
Conversation,Fillerwords,85,"El porcentaje de muletillas empleadas es 1.55%, siendo las muletillas mas repetidas: mire, va"
Conversation,Clarity,60,"Deberías enfocarte en mantener ideas estructuradas y claras en tus explicaciones. Usa términos específicos y evita frases demasiado largas que puedan diluir el mensaje. Por ejemplo, en lugar de decir 'las curvas sinérgicas', explica qué beneficios concretos aportan. Además, mantén un tono profesional y evita expresiones coloquiales para asegurar que el mensaje resulte más formal y directo. Simplifica las ideas complejas y revisa que cada frase tenga un objetivo claro y cerrado."
Conversation,Participation,90,Tu porcentaje de participación ha sido del 34.22%. Has mostrado escucha activa en 4 ocasiones.
Conversation,Key Themes,100,"Has cubierto todos los temas clave esenciales de calidad de soldadura, precisión técnica y cumplimiento normativo. Excelente trabajo en desarrollar cada uno con argumentos específicos."
Conversation,Index of Questions,0,Métrica desactivada temporalmente
Conversation,Rhythm,20,Velocidad de habla demasiado lenta (81.4 PPM). Intenta aumentar el ritmo para mantener la atención del cliente.
Conversation,Objective Accomplished,Yes,-
Profile,Prospection,5,"Demuestras investigación especifica al conocer detalles técnicos y necesidades del cliente, como 'certificación ISO' y 'problemas con aluminio', además de ofrecer soluciones adaptadas, como 'calibración y verificación'."
Profile,Empathy,5,"No interrumpes y validas las preocupaciones del cliente con frases como 'Eso suena interesante' y 'Eso es excelente', mostrando empatía y comprensión profunda, además de reforzar sus inquietudes. Utilizas un lenguaje empático y confirmas necesidades específicas."



🎯 CONVERSATION SCORES:


Category,Metric,Score,Feedback
Conversation,General Score,88.5,-
Conversation,Fillerwords,85,"El porcentaje de muletillas empleadas es 1.55%, siendo las muletillas mas repetidas: mire, va"
Conversation,Clarity,60,"Deberías enfocarte en mantener ideas estructuradas y claras en tus explicaciones. Usa términos específicos y evita frases demasiado largas que puedan diluir el mensaje. Por ejemplo, en lugar de decir 'las curvas sinérgicas', explica qué beneficios concretos aportan. Además, mantén un tono profesional y evita expresiones coloquiales para asegurar que el mensaje resulte más formal y directo. Simplifica las ideas complejas y revisa que cada frase tenga un objetivo claro y cerrado."
Conversation,Participation,90,Tu porcentaje de participación ha sido del 34.22%. Has mostrado escucha activa en 4 ocasiones.
Conversation,Key Themes,100,"Has cubierto todos los temas clave esenciales de calidad de soldadura, precisión técnica y cumplimiento normativo. Excelente trabajo en desarrollar cada uno con argumentos específicos."
Conversation,Index of Questions,0,Métrica desactivada temporalmente
Conversation,Rhythm,20,Velocidad de habla demasiado lenta (81.4 PPM). Intenta aumentar el ritmo para mantener la atención del cliente.
Conversation,Objective Accomplished,Yes,-



👥 PROFILE SCORES:


Category,Metric,Score,Feedback
Profile,Prospection,5,"Demuestras investigación especifica al conocer detalles técnicos y necesidades del cliente, como 'certificación ISO' y 'problemas con aluminio', además de ofrecer soluciones adaptadas, como 'calibración y verificación'."
Profile,Empathy,5,"No interrumpes y validas las preocupaciones del cliente con frases como 'Eso suena interesante' y 'Eso es excelente', mostrando empatía y comprensión profunda, además de reforzar sus inquietudes. Utilizas un lenguaje empático y confirmas necesidades específicas."
Profile,Technical Domain,4,"El vendedor menciona 'el sistema del arrastre de cuatro ruletas que garantiza una velocidad de hidro constante de veinticuatro metros al minuto', demostrando conocimiento técnico con datos específicos. Sin embargo, no profundiza en detalles más complejos o comparaciones. Por ello, cumple parcialmente con los criterios para un dominio avanzado."
Profile,Negotiation,5,"El vendedor justifica el valor destacando beneficios técnicos, como 'reduce proyecciones y retrabajo' y 'garantiza cumplimiento de normativa', y propone cerrar con un presupuesto, sin ofrecer descuentos inmediatos ni ceder en las objeciones, proponiendo pasos concretos como enviar una propuesta formal."
Profile,Resilience,5,"El vendedor mantiene un tono positivo y enérgico, responde con seguridad a las objeciones del cliente, recupera la energía tras cada respuesta (como en 'La estabilidad es cojonuda') y termina con un acuerdo claro, solicitando envío de propuesta formal. No se detectan signos de frustración ni cambios negativos en su actitud."


## Stress Test: LLM Scoring Consistency

This cell runs the scoring and profiling functions **5 times** to test the consistency and variance of LLM-based scoring. This helps identify how much variation exists in the scores due to the non-deterministic nature of LLMs.

In [19]:
all_conversation_runs, all_profile_runs = await stress_test(iterations=10, transcript=transcript)

🧪 STRESS TEST: Computing scores 5 times...
This will help us understand the variance in LLM-based scoring.

🔄 Run 1/10...
   ✅ Completed in 22.78s

🔄 Run 2/10...
   ✅ Completed in 20.59s

🔄 Run 3/10...
   ✅ Completed in 25.50s

🔄 Run 4/10...
llamando a gpt otra vez porque no daba un JSON bien formado... (intento 1/10)
   ✅ Completed in 27.94s

🔄 Run 5/10...
   ✅ Completed in 21.11s

🔄 Run 6/10...
   ✅ Completed in 20.72s

🔄 Run 7/10...
   ✅ Completed in 23.83s

🔄 Run 8/10...
   ✅ Completed in 21.61s

🔄 Run 9/10...
   ✅ Completed in 20.17s

🔄 Run 10/10...
llamando a gpt otra vez porque no daba un JSON bien formado... (intento 1/3)
   ✅ Completed in 22.32s

✅ All 5 runs completed!


In [20]:
display_stress_test(all_conversation_runs, all_profile_runs)


📋 ALL RUNS - CONVERSATION SCORES


run,general_score,fillerwords_scoring,clarity_scoring,participation_scoring,keythemes_scoring,indexofquestions_scoring,rhythm_scoring,is_accomplished
1,88.5,85,60,90,100,0,20,True
2,88.5,85,60,90,100,0,20,True
3,90.0,85,80,90,100,0,20,True
4,89.2,85,70,90,100,0,20,True
5,88.5,85,60,90,100,0,20,True
6,90.0,85,80,90,100,0,20,True
7,88.5,85,60,90,100,0,20,True
8,88.5,85,60,90,100,0,20,True
9,89.2,85,70,90,100,0,20,True
10,89.2,85,70,90,100,0,20,True



📋 ALL RUNS - PROFILE SCORES


run,prospection_scoring,empathy_scoring,technical_domain_scoring,negotiation_scoring,resilience_scoring
1,5,5,4,5,5
2,5,5,5,5,5
3,5,5,4,5,5
4,5,5,4,5,5
5,5,5,4,5,5
6,5,5,4,4,5
7,5,5,4,5,5
8,5,5,4,4,5
9,5,5,4,5,5
10,5,5,4,5,5
